In [4]:
!pip install transformers==3
!pip install tensorflow-gpu
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import os
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import BertModel, BertTokenizer
import warnings
warnings.filterwarnings('ignore')
# specify GPU
device = torch.device("cuda")
print(device)
# Chose Language
language="it-IT"
berts={"it-IT":"bert-base-multilingual-cased", "en-US":"bert-base-uncased"}
class_type="grounding"
classifications={"grounding":"Grounding_Dataset", "learning":"Learning_Dataset"}

cuda


In [268]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [269]:
data_dir = "/content/drive/My Drive/Tesi/Code/Intent_classification"

In [270]:
class LoadingData():
            
    def __init__(self):
        #Davide="Ingegneria/Magistrale/Tesi/"
        train_file_path = os.path.join(data_dir,classifications[class_type],"Train")
        validation_file_path = os.path.join(data_dir,classifications[class_type],"Validate")
        category_id = 0
        self.cat_to_intent = {}
        self.intent_to_cat = {}
        print(train_file_path)
        for dirname, _, filenames in os.walk(train_file_path):
            for filename in filenames:
                
                file_path = os.path.join(dirname, filename)
                intent_id = filename.replace(".txt","")
                self.cat_to_intent[category_id] = intent_id
                self.intent_to_cat[intent_id] = category_id
                category_id+=1
        '''Training data'''
        self.train_data_frame = self.file_to_dataframe(train_file_path)  
        self.train_data_frame = self.train_data_frame.sample(frac = 1)
        
        '''Validation data''' 
        self.validation_data_frame = self.file_to_dataframe(validation_file_path)   
        self.validation_data_frame = self.validation_data_frame.sample(frac = 1)

    def file_to_dataframe(self,file_path,file_path_2=None):
          data = list() 
          for dirname, _, filenames in os.walk(file_path):
              for filename in filenames:
                  file_path = os.path.join(dirname, filename)
                  intent_id = filename.replace(".txt","")
                  data+=self.make_data_for_intent_from_txt(file_path,intent_id,self.intent_to_cat[intent_id])
          if file_path_2!=None:
              for dirname, _, filenames in os.walk(file_path_2):
                  for filename in filenames:
                      file_path_2 = os.path.join(dirname, filename)
                      intent_id = filename.replace(".txt","")
                      data+=self.make_data_for_intent_from_txt(file_path_2,intent_id,self.intent_to_cat[intent_id])      
          return pd.DataFrame(data, columns =['query', 'intent', 'category'])  
        
        
    def make_data_for_intent_from_txt(self,txt_file,intent_id,cat):
        query_list=[]
        with open(txt_file, 'r') as f:
            for line in f.readlines():
                text=line.replace("\n", "")
                query_list.append((text,intent_id,cat))
        return query_list 

In [271]:
ld = LoadingData()

/content/drive/My Drive/Tesi/Code/Intent_classification/Grounding_Dataset/Train


In [272]:
train_df = ld.train_data_frame

In [273]:
label_map,id2label = ld.intent_to_cat,ld.cat_to_intent

In [274]:
train_text, val_text, train_labels, val_labels = train_test_split(train_df['query'], train_df['category'], 
                                                                    random_state=2018, 
                                                                    test_size=0.2, 
                                                                    stratify=train_df['category'])

In [275]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(berts[language])
bert = BertModel.from_pretrained(berts[language])

In [10]:
seq_len = [len(i.split()) for i in train_text]

pd.Series(seq_len).hist(bins = 30)
max_seq_len = max(seq_len)
print(max_seq_len)

NameError: ignored

In [277]:
# tokenize and encode sequences in the training set
if max_seq_len>512:
    max_seq_len = 512
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)



In [278]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())
print("train_y:",train_y)
# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())
print("val_y:",val_y)


train_y: tensor([1, 1, 5, 0, 2, 1, 0, 3, 2, 0, 3, 1, 3, 5, 1, 0, 3, 2, 3, 3, 0, 0, 3, 2,
        2, 3, 1, 2, 2, 2, 3, 1, 3, 2, 2, 3, 0, 2, 1, 2, 0, 0, 2, 3, 1, 0, 1, 0,
        3, 0, 3, 2, 1, 1, 1, 0, 3, 3, 0, 5, 0, 3, 0, 2, 2, 2, 3, 0, 3, 3, 2, 1,
        4, 3, 2, 3, 2, 2, 2, 0, 3, 3, 1, 1, 3, 1, 2, 1, 0, 5, 1, 1, 0, 0, 2, 1,
        0, 2, 3, 3, 0, 1, 0, 3, 2, 3, 3, 1, 1, 0, 3, 1, 2, 0, 3, 0, 2, 0, 1, 1,
        0, 2, 0, 1, 1, 2, 0, 3, 1, 0, 3, 0, 0, 2, 0, 1, 1, 2, 3, 0, 1, 0, 3, 0,
        1, 2, 3, 4, 0, 0, 0, 3, 1, 3, 2, 3, 3, 3, 1, 0, 2, 1, 3, 4, 4, 0, 3, 4,
        2, 1, 3, 1, 0, 2, 2, 0, 0, 2, 2, 0, 0, 1, 1, 1, 4, 0, 1, 2, 1, 0, 5, 3,
        0, 2, 1, 3, 5, 2, 0, 1, 0, 0, 0, 1, 2, 2, 2, 2, 3, 5, 3, 2, 3, 1, 2, 1,
        2, 2, 1, 2, 2, 0, 1, 0, 3, 0, 5, 3, 3, 2, 5, 0, 0, 3, 2, 3, 0, 2, 1, 3,
        0, 0, 1, 2, 0, 0, 2, 0, 3, 2, 0, 1, 0, 1, 0, 0, 4, 5, 3, 4, 0, 1, 1, 2,
        0, 1, 4, 3, 0, 2, 3, 2, 0, 2, 3, 1, 2, 1, 4, 3, 1, 3, 1, 1, 3, 0, 1, 1,
        3, 0, 2, 2, 1, 0, 2, 0,

In [279]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 16

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [280]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [8]:
class BERT_Arch(nn.Module):
    def __init__(self, bert,label_map):
        super(BERT_Arch, self).__init__()
        self.bert = bert 
      
        # dropout layer
        self.dropout = nn.Dropout(0.1)

        # relu activation function
        self.relu =  nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768,512)

        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,len(label_map))

        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

        #pass the inputs to the model  
        _, cls_hs = self.bert(sent_id, attention_mask=mask)

        x = self.fc1(cls_hs)

        x = self.relu(x)

        x = self.dropout(x)

        # output layer
        x = self.fc2(x)

        # apply softmax activation
        x = self.softmax(x)
        return x

In [282]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert,label_map)

# push the model to GPU
model = model.to(device)

In [283]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)

In [284]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_wts = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print(class_wts)

[0.61666667 0.77083333 0.77083333 0.71153846 5.28571429 5.28571429]


In [285]:
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 60

In [286]:
# function to train the model
def train():
    model.train()

    total_loss, total_accuracy = 0, 0
  
    # empty list to save model predictions
    total_preds=[]
    total_labels =[]
  
    # iterate over batches
    for step,batch in enumerate(train_dataloader):
    
        # progress update after every 50 batches.
        if step % 100 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        # push the batch to gpu
        batch = [r.to(device) for r in batch]

        sent_id, mask, labels = batch

        # clear previously calculated gradients 
        model.zero_grad()        

        # get model predictions for the current batch
        preds = model(sent_id, mask)

        # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)

        # add on to the total loss
        total_loss = total_loss + loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # model predictions are stored on GPU. So, push it to CPU
        preds = preds.detach().cpu().numpy()
        preds = np.argmax(preds, axis=1)
        # append the model predictions
        total_preds+=list(preds)
        total_labels+=labels.tolist()

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    #total_preds  = np.concatenate(total_preds, axis=0)
    f1 = f1_score(total_labels, total_preds, average='weighted')
    #returns the loss and predictions
    return avg_loss, f1

In [287]:
# function for evaluating the model
def evaluate():
  
    print("\nEvaluating...")

    # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0

    # empty list to save the model predictions
    total_preds = []
    total_labels = []
    # iterate over batches
    for step,batch in enumerate(val_dataloader):
    
        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:

          # Calculate elapsed time in minutes.
          #elapsed = format_time(time.time() - t0)

          # Report progress.
          print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

        # push the batch to gpu
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

        # deactivate autograd
        with torch.no_grad():

            # model predictions
            preds = model(sent_id, mask)

            # compute the validation loss between actual and predicted values
            loss = cross_entropy(preds,labels)

            total_loss = total_loss + loss.item()

            preds = preds.detach().cpu().numpy()
            preds = np.argmax(preds, axis=1)
            total_preds+=list(preds)
            total_labels+=labels.tolist()
    # compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader) 

    # reshape the predictions in form of (number of samples, no. of classes)
    #total_preds  = np.concatenate(total_preds, axis=0)
    
    f1 = f1_score(total_labels, total_preds, average='weighted')
    return avg_loss, f1

In [288]:
def save_checkpoint(filename, epoch, model, optimizer, label_map, id2label):
    state = {
        'epoch': epoch,
        'model': model,
        'optimizer': optimizer,
        'label_map': label_map,
        'id_map':id2label}
    torch.save(state, filename)

In [289]:
# set initial loss to infinite
best_valid_loss = float('inf')
best_valid_f1 = float('-inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, f1_train = train()
    
    #evaluate model
    valid_loss, f1_valid = evaluate()

    #save the best f1
    if f1_valid > best_valid_f1:
        print('\nNEW BEST VALID F1 {:.3f} Valid Loss {:.3f}  Epoch  {:}'.format(best_valid_f1, valid_loss, epoch + 1))
        best_valid_f1 = f1_valid
        file_name = class_type+'_saved_weights_f1.pt'
        save_checkpoint(file_name, epoch, model, optimizer, label_map, id2label)

    #save the best model
    if valid_loss < best_valid_loss:
        print('\nNEW BEST VALID LOSS {:.3f} Valid_f1 {:.3f}  Epoch  {:}'.format(best_valid_loss, f1_valid, epoch + 1))
        best_valid_loss = valid_loss
        file_name = class_type+'_saved_weights_loss.pt'
        save_checkpoint(file_name, epoch, model, optimizer, label_map, id2label)
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')
    print(f'\nTraining F1: {f1_train:.3f}')
    print(f'Validation F1: {f1_valid:.3f}')

print('\n BEST VALID LOSS FOUND {:.3f} BEST VALID F1 FOUND {:.3f}'.format(best_valid_loss, best_valid_f1))


 Epoch 1 / 60

Evaluating...

NEW BEST VALID F1 -inf Valid Loss 1.845  Epoch  1

NEW BEST VALID LOSS inf Valid_f1 0.348  Epoch  1

Training Loss: 1.809
Validation Loss: 1.845

Training F1: 0.227
Validation F1: 0.348

 Epoch 2 / 60

Evaluating...

NEW BEST VALID F1 0.348 Valid Loss 1.556  Epoch  2

NEW BEST VALID LOSS 1.845 Valid_f1 0.389  Epoch  2

Training Loss: 1.714
Validation Loss: 1.556

Training F1: 0.355
Validation F1: 0.389

 Epoch 3 / 60

Evaluating...

NEW BEST VALID F1 0.389 Valid Loss 1.571  Epoch  3

Training Loss: 1.641
Validation Loss: 1.571

Training F1: 0.433
Validation F1: 0.409

 Epoch 4 / 60

Evaluating...

NEW BEST VALID LOSS 1.556 Valid_f1 0.358  Epoch  4

Training Loss: 1.567
Validation Loss: 1.457

Training F1: 0.419
Validation F1: 0.358

 Epoch 5 / 60

Evaluating...

NEW BEST VALID F1 0.409 Valid Loss 1.399  Epoch  5

NEW BEST VALID LOSS 1.457 Valid_f1 0.506  Epoch  5

Training Loss: 1.497
Validation Loss: 1.399

Training F1: 0.437
Validation F1: 0.506

 Epoch

In [290]:
path = class_type+'_saved_weights_loss.pt' #!!!!!
test_df = ld.validation_data_frame

checkpoint = torch.load(path, map_location=device)
model = checkpoint.get("model")

tokenizer = BertTokenizer.from_pretrained(berts[language])

# weights_path = os.path.join(data_dir,"Weights","topic_saved_weights.pt")
!cp learning_saved_weights.pt /content/drive/My\ Drive/Tesi/Code/Intent_classification/Weights
!cp grounding_saved_weights.pt /content/drive/My\ Drive/Tesi/Code/Intent_classification/Weights

# tokenize and encode sequences in the test set
test_text,test_labels = test_df["query"],test_df["category"]

tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())
print("test_y:",test_y)

cp: cannot stat 'learning_saved_weights.pt': No such file or directory
cp: cannot stat 'grounding_saved_weights.pt': No such file or directory
test_y: tensor([0, 0, 3, 3, 0, 0, 2, 2, 0, 5, 2, 3, 1, 4, 1, 0, 3, 2, 0, 5, 2, 2, 1, 0,
        1, 1, 0, 2, 3, 2, 3, 2, 4, 0, 3, 2, 2, 1, 2, 1, 2, 0, 1, 2, 2, 1, 2, 3,
        2, 2, 3, 2, 2, 2, 2, 3, 4, 3, 0, 4, 2, 2, 3, 2, 3, 2, 2, 5, 5, 1, 2, 3,
        3, 1, 1, 2, 5, 0, 2, 5, 1, 1, 4, 1, 0, 0])


In [291]:
# get predictions for test data
with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()

In [292]:
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.75      0.80      0.77        15
           1       1.00      0.80      0.89        15
           2       1.00      0.93      0.97        30
           3       0.88      1.00      0.94        15
           4       1.00      1.00      1.00         5
           5       0.75      1.00      0.86         6

    accuracy                           0.91        86
   macro avg       0.90      0.92      0.90        86
weighted avg       0.92      0.91      0.91        86



In [13]:
max_seq_len=9 #!|!!

class Prediction:
    def __init__(self):
        path = class_type+'_saved_weights_f1.pt'  #!!!!!

        checkpoint = torch.load(path,map_location=device)
        self.predictor = checkpoint.get("model")
        self.tokenizer = BertTokenizer.from_pretrained(berts[language])
        self.tag = checkpoint.get("id_map")

    def predict(self,text):
        tokens = self.tokenizer.tokenize(text)
        tokens = tokens[:max_seq_len - 2]
        tokens = ['[CLS]'] + tokens + ['[SEP]']

        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        input_ids = input_ids + [0] * (max_seq_len-len(input_ids))
        input_ids = torch.tensor(input_ids).unsqueeze(0)
        input_ids = input_ids.to(device)

        input_mask = [1]*len(tokens) + [0] * (max_seq_len - len(tokens))
        input_mask = torch.tensor(input_mask).unsqueeze(0)
        input_mask = input_mask.to(device)

        logits = self.predictor(input_ids,input_mask)
        prob = torch.nn.functional.softmax(logits,dim=1)
        result = [(self.tag[idx],item *100) for idx,item in enumerate(prob[0].tolist())]
        preds = logits.detach().cpu().numpy()
        pred_val = np.argmax(preds)
        pred_val = self.tag[pred_val]
        return result,pred_val


In [14]:
pred = Prediction()

print(max_seq_len)

9


In [18]:
list_input = [
     'di che colore è',
     'che tessitura ha questo oggetto',
     'che cosa è questo',
     'che forma ha',
     'cosa è',
     'che superficie ha questo oggetto',
     'texture',
     'è un toroide',
     'abbandona',
     'ammazzati',
     'ti posso insegnare qualcosa',
     'Petrucci'     
     ] 

for item in list_input:
    confidence,pred_val = pred.predict(item)
    prob = round([i for i in confidence if i[0]==pred_val][0][1],4)
    print("'" + item + "' = " + pred_val + ": " + str(prob))
    print([(i[0],round(i[1],4)) for i in confidence])
    print()

'di che colore è' = color_query: 98.1084
[('texture_query', 1.1794), ('shape_query', 0.6634), ('general_query', 0.0488), ('color_query', 98.1084), ('exit', 0.0), ('training_mode', 0.0001)]

'che tessitura ha questo oggetto' = texture_query: 72.2735
[('texture_query', 72.2735), ('shape_query', 2.3775), ('general_query', 4.4984), ('color_query', 20.6624), ('exit', 0.0893), ('training_mode', 0.0989)]

'che cosa è questo' = general_query: 98.9833
[('texture_query', 0.8596), ('shape_query', 0.0425), ('general_query', 98.9833), ('color_query', 0.0821), ('exit', 0.0043), ('training_mode', 0.0282)]

'che forma ha' = shape_query: 99.2984
[('texture_query', 0.5004), ('shape_query', 99.2984), ('general_query', 0.059), ('color_query', 0.1421), ('exit', 0.0001), ('training_mode', 0.0)]

'cosa è' = general_query: 98.9998
[('texture_query', 0.8754), ('shape_query', 0.0195), ('general_query', 98.9998), ('color_query', 0.069), ('exit', 0.0042), ('training_mode', 0.0321)]

'che superficie ha questo ogge

In [5]:
!cp grounding_saved_weights_loss.pt /content/drive/My\ Drive/Tesi/Code/Intent_classification/Weights
!cp grounding_saved_weights_f1.pt /content/drive/My\ Drive/Tesi/Code/Intent_classification/Weights